<a href="https://colab.research.google.com/github/josephasal/cosmo_inference/blob/diagnostics-%2B-adaptive-sampling/mcmc/adaptive_sampling_mcmc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#Contains the core code for the MCMC algorithm implementated
#Have now made this one use adaptive samplling, adjusting the proposal as it goes along

#Dimensionless distance modulus function implementation
import numpy as np
#Distance modulus function
def calculate_distance_modulus(z, omega_m,h):
  """
  Calculates dimensionless theoretical distance modulus using

  inputs:
   - z: Redshift
   - omega_m: density matter parameter
   - h: dimensionless hubble constance H0 = 100h km/s/Mpc

   outputs: theoretical distance modulus
  """
  c = 299792.458   # speed of light in km/s
  H0 = 100 * h     # Hubble constant in km/s/Mpc

  #Luminosity distance based on Penn 1999 analytic solution

  #Fitting function
  def eta(a,omega_m):
    """
    Fits eta
    inputs:
      a - a number
      omega_m - matter density

    outputs: eta as a function of a and omega_m
    """
    s = ((1-omega_m)/omega_m)**(1/3)
    eta = 2*np.sqrt(s**3 +1) * ((1/(a**4)) - 0.1540*(s/(a**3)) + 0.4304 *((s**2)/(a**2)) + 0.19097*((s**3)/a) + 0.066941*(s**4))**(-1/8)

    return eta

  #Calculate eta for 1 and 1/z+1
  a = 1/(z+1)
  eta_1 = eta(1,omega_m)
  eta_z = np.array([eta(ai, omega_m) for ai in a])

  #Dimensionless luminosity distance calculation
  d_L_star = (c/H0) * (1+z) * (eta_1 - eta_z)



  #Now to calculate distance modulus mu
  theoretical_mu = 25 - 5*np.log10(h) + 5*np.log10(d_L_star)
  return theoretical_mu

In [11]:
#MCMC basic implementation

#Likelihood function, standard gaussian function

def log_likelihood(mu_obs, mu_model, sigma_mu):

  """
  Computes thes logged likelihood for the sample and observed distance modulus

  inputs:
    - mu_obs: observed mu (mu from the data)
    - mu_model: theoretical mu from the model
    - sigma_mu: standard deviation of the observed mu (uncertainty)

  outputs:
    - log likelihood

  """
  return -0.5 * np.sum((mu_obs - mu_model)**2/sigma_mu**2)


# Defining the prior as a function
def log_prior(params):
  """
  Function that sets a uniform prior of omega m and h

  """
  omega_m, h = params
  if 0.1 < omega_m < 0.5 and 0.4 < h < 0.9:
    return 0.0

  else:
    return -np.inf #acceptance probability is 0 if not in the priors upper and lower bounds



#Metropolis Hastings algorithm

def adaptive_metropolis_hastings(likelihood, z, mu_obs, sigma_mu, n_steps, initial_params, step_size, burn_in, n_walkers):
  """
  Perform Metropolis Hastings MCMC to sample from the posterior

  inputs:
    - likelihood: function to compute the likelihood
    - z: redshift
    - mu_obs: observed mu
    - sigma_mu: standard deviation of the observed mu
    - n_steps: Number of steps for MCMC
    - intial_params: initial guesses for [omega_m, h] for each walker, has to be array with rows = number of walkers
    - step size: proposal step size for [omega_m, h]
    - burn_in : percentage of chain to discard for the burn in period (given as a decimal)
    - n_walkers: number of walkers that are sampling

  outputs:
  Array of 3 dimensions, in shape of (steps after burn in , walker number, 2)

  """
  params = np.array(initial_params) #input as [] bracket so just make an array
  samples = []
  accepted_samples = np.zeros(n_walkers) #gonna use to calculate acceptance rate

  #New adaptive parameters:
  update_interval = 100  #update every 100 iterations
  target_alpha = 0.25 #set a target acceptance rate of 25%
  learning_rate = 0.1 #adaptation speed parameter
  accepted_cycle = np.zeros(n_walkers)  #empty array of accepted proposals at each cycle/iteration


  for step in range (n_steps):
    new_params = np.empty_like(params) #empty array same size as intial parameters array, need to keep track of new paramters for each walker via matrix

    #Do similar calulation for new parameters as before but just have to loop it for all walkers now
    for i in range(n_walkers):

      #Proposal
      #guess for new parameters, propsal distribution is a multivariate gaussian distribution now
      #Step size has to be a 2x1 array
      sigma_omega_m = step_size[0]
      sigma_h = step_size[1]

      #covariance matrix thing off diagonals are the correlation between the values
      rho = 0 #set as 0 intially, alogrithm will see if there are correlations --> explores parameter space better
      covariance_matrix = np.array([[sigma_omega_m**2, rho * sigma_omega_m * sigma_h], [rho * sigma_omega_m * sigma_h, sigma_h**2]])

      #drawing from this new proposal now but for each walker/ walker i
      proposed_params = params[i] + np.random.multivariate_normal(np.zeros(2), covariance_matrix)
      omega_m_proposed, h_proposed = proposed_params

      # Priors on Omega_m and h, using our new functions
      log_prior_proposed = log_prior(proposed_params)

      #If prior is  - infininty then proposal out of bounds so reject
      if np.isneginf(log_prior_proposed):
        new_params[i] = params[i]
        continue

      #Distance modulus and log likelihood of proposed parameters
      proposed_mu_model = calculate_distance_modulus(z, proposed_params[0], proposed_params[1]) #now for each walker
      proposed_log_likelihood = log_likelihood(mu_obs, proposed_mu_model, sigma_mu)


      #Posterior for proposed parameters
      proposed_log_posterior = proposed_log_likelihood + log_prior_proposed

      #Distance modulus, log likelihood and posterior of current parameters, initially inputted from the function
      current_mu_model = calculate_distance_modulus(z, params[i,0], params[i,1])  #now for each walker
      current_log_likelihood = log_likelihood(mu_obs, current_mu_model, sigma_mu)
      current_log_posterior = current_log_likelihood + log_prior(params[i])

      #Calculate the acceptance probability, now based on log posteriors
      delta_log_posterior = proposed_log_posterior - current_log_posterior

      #Implementing explicit overflow protection
      #If the difference in likelihoods is at max python limit, then accept the new proposal with probaiblity 1. Means new parameters are leng
      if delta_log_posterior > 700:
        acceptance_probability = 1.0

      #If difference in likelihood is at min python limit, then dont accept the new proposal at all. Means new parameters are clapped
      elif delta_log_posterior < -700:
        acceptance_probability = 0.0

      #If difference something else then we accept with probability below and randomly sample. Lets us explore parameter space
      else:
        acceptance_probability = min(1, np.exp(delta_log_posterior))


      u = np.random.uniform(0,1) #set the randomness part of accept/ reject

      #Accept proposed move
      if u < acceptance_probability:
        new_params[i] = proposed_params
        accepted_samples[i] += 1
        accepted_cycle[i] += 1 #also add acceptance count for each walker adaptation thing
      else:
        new_params[i] = params[i] #chain doesnt move and retrys sampling

    #Updating all the walkers at the same time, outside of the loop, do end of every loop
    params = new_params.copy()
    samples.append(params.copy())

    #Adaptive update after every update interval iteration loop
    if (step +1) % update_interval == 0: #check if nth+1 iteration is exact multiple, so saying it has don 100/200/300 steps
      average_acceptance = np.mean(accepted_cycle)/ update_interval #acceptance rate of this cycle (100 steps)

      #Increase step size of alpha larger than target, decrease step size if alpha is too small
      #Robbins Munro approximation/ Haario et al.
      scale_factor = np.exp(learning_rate * (average_acceptance - target_alpha))

      #Now scale both step sizes for omega_m and h by this scale factor
      step_size = [step_size[0]* scale_factor, step_size[1]* scale_factor] #list comprehension innit

      #Something to tell me what is happening
      print(f"After {step+1} iteration alpha = {average_acceptance}, new step size = {step_size}")

      #Reset counter
      accepted_cycle = np.zeros(n_walkers)

  #Acceptance ratio calculation
  acceptance_ratio = accepted_samples/ n_steps
  print(f"MCMC carried out with {n_steps} steps, and acceptance ratio of each walker {acceptance_ratio}")


  #Number of samples to discard from the chain due to burn in
  burned_chains = int(burn_in * n_steps) #keep integer
  samples_post_burn = samples[burned_chains:] #use everything after the burn in number

  return np.array(samples_post_burn)



